In [ ]:
import os
import pickle
import blosc
import matplotlib.pyplot as plt
import numpy as np
from coincidences_analyzer import calculate_coincidences, cfg

def decompress_data_blosc(compressed_data):
    """
    Decompresses the data structure using blosc.
    """
    # Decompress and deserialize
    serialized_data = blosc.decompress(compressed_data)
    data = pickle.loads(serialized_data)
    return data

def open_pickle_file(file_path):
    """
    Opens a single pickle file and returns its content.
    
    :param file_path: Path to the pickle file.
    :return: Loaded pickle data.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    with open(file_path, 'rb') as file:
        return pickle.load(file)


def analyze_coincidences(alice_dir, bob_dir, start_epoch, num_seconds, config):
    """
    Parses pickle files from Alice and Bob, finds coincidences, and returns the results.
    
    :param alice_dir: Directory containing Alice's data.
    :param bob_dir: Directory containing Bob's data.
    :param start_epoch: Starting epoch time (integer, ignoring fractions).
    :param num_seconds: Number of seconds to analyze.
    :param decompress_func: Function to decompress data.
    :return: List of (epoch_time, coincidence count).
    """
    results = []
    
    for i in range(num_seconds):
        epoch_time = start_epoch + i
        
        alice_files = [f for f in os.listdir(alice_dir) if f.startswith(str(epoch_time)) and f.endswith('.pkl')]
        bob_files = [f for f in os.listdir(bob_dir) if f.startswith(str(epoch_time)) and f.endswith('.pkl')]
        
        if alice_files and bob_files:
            alice_data = decompress_data_blosc(open_pickle_file(os.path.join(alice_dir, alice_files[0])))
            bob_data = decompress_data_blosc(open_pickle_file(os.path.join(bob_dir, bob_files[0])))
            coincidences = calculate_coincidences(config, alice_data, bob_data)
            results.append((epoch_time, coincidences))
        else:
            results.append((epoch_time, 0))
    
    return results


def plot_coincidences(results):
    """
    Plots the coincidence analysis results.
    
    :param results: List of (epoch_time, coincidence count).
    """
    times, coincidence_counts = zip(*results)
    
    plt.figure(figsize=(10, 5))
    plt.plot(times, coincidence_counts, marker='o', linestyle='-')
    plt.xlabel("Epoch Time (s)")
    plt.ylabel("Coincidence Count")
    plt.title("Coincidence Analysis Between Alice and Bob")
    plt.xticks(rotation=45)
    plt.grid()
    plt.show()

In [ ]:
config = cfg(peak0=0,
             range_ns=10,
             time_bin=0.01,
             time_integration_ms=None,
             low_coinc_windows=None,
             high_coinc_windows=None
             )

# Example usage
alice_data_path = 'data/2/'
bob_data_path = 'data/2/'
start_epoch = 1741698796  # Starting second
num_seconds = 10  # Number of next seconds to analyze
results = analyze_coincidences(alice_data_path, bob_data_path, start_epoch, num_seconds, config)
plot_coincidences(results)